https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-9-neural-networks-with-tfidf-vectors-using-d0b4af6be6d7

In [21]:
import numpy as np
import pandas as pd


np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re
from sklearn.model_selection import validation_curve
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
#from textblob import TextBlob
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from wordcloud import WordCloud
from utils import open_by_tweets
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time
from utils import create_csv_submission
SEED = 15432

In [87]:
train_set = pd.read_pickle("./data/tweets.pkl")
del train_set['tweet_len']


x = train_set.tweet
y = train_set.label

x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)
#x_validation_tfidf, x_test, y_validation, y_test = train_test_split(x_test, y_test, test_size=.5, random_state=SEED)


tvec = TfidfVectorizer(max_features=70000,ngram_range=(1, 3))
x_train_tfidf = tvec.fit_transform(x_train)
x_validation_tfidf = tvec.transform(x_validation)



In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=70000))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [89]:
model.fit(x_train_tfidf,y_train, batch_size =  32, epochs = 6)

Epoch 1/6
177694/177694 [==============================] - 780s 4ms/step - loss: 0.4128 - accuracy: 0.8057
Epoch 2/6
177694/177694 [==============================] - 516s 3ms/step - loss: 0.3028 - accuracy: 0.8647
Epoch 3/6
177694/177694 [==============================] - 539s 3ms/step - loss: 0.2343 - accuracy: 0.8981
Epoch 4/6
177694/177694 [==============================] - 543s 3ms/step - loss: 0.1726 - accuracy: 0.9279
Epoch 5/6
177694/177694 [==============================] - 557s 3ms/step - loss: 0.1171 - accuracy: 0.9539
Epoch 6/6
177694/177694 [==============================] - 562s 3ms/step - loss: 0.0749 - accuracy: 0.9720


In [90]:
to_predict = pd.read_pickle("./data/tweets_test.pkl")
del to_predict['tweet_len']

to_predict.index += 1

to_predict = to_predict['tweet']
to_predict = to_predict.astype(str)

to_predict = tvec.transform(to_predict)





In [91]:
result = model.predict(to_predict)
result
#it returns values between [0,1] (since sigmoid is used) 
#result[result < 0.5] = -1 #replace values < 0.5 to -1
#result[result >= 0.5] = 1

array([[0.0000000e+00],
       [1.4980549e-01],
       [8.0309367e-01],
       ...,
       [0.0000000e+00],
       [9.2596138e-01],
       [2.1368265e-05]], dtype=float32)

In [93]:
result[result < 0.5] = -1 #replace values < 0.5 to -1
result[result >= 0.5] = 1
result

array([[-1.],
       [-1.],
       [ 1.],
       ...,
       [-1.],
       [ 1.],
       [-1.]], dtype=float32)

In [94]:

create_csv_submission([x for x in range(1,len(result)+1)],result,"keras.csv")